## Importing Libraries

In [ ]:
import tensorflow as tf

In [ ]:
a=[[[1,2,3,4],
 [5,6,7,8]]]
tf.argmax(a,axis=-1)

<tf.Tensor: shape=(1, 2), dtype=int64, numpy=array([[3, 3]])>

## Importing Data

In [ ]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2023-06-24 09:44:14--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  33.5MB/s    in 0.2s    

2023-06-24 09:44:15 (33.5 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]



In [ ]:
!unzip "/content/fra-eng.zip" -d "/content/fra"

Archive:  /content/fra-eng.zip
  inflating: /content/fra/_about.txt  
  inflating: /content/fra/fra.txt    


In [ ]:
dataset=tf.data.TextLineDataset("/content/fra/fra.txt")

In [ ]:
for i in dataset.take(5):
  print(i)

tf.Tensor(b'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)', shape=(), dtype=string)
tf.Tensor(b'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)', shape=(), dtype=string)
tf.Tensor(b'Go.\tEn route !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)', shape=(), dtype=string)
tf.Tensor(b'Go.\tBouge !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)', shape=(), dtype=string)
tf.Tensor(b'Hi.\tSalut !\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)', shape=(), dtype=string)


In [ ]:
len(list(dataset))# total number of lines/documents

217975

In [ ]:
for i in dataset.skip(217974).take(1):
  print(i)
  print(len(tf.strings.split(i," "))) # len of last line/document to get the sequence length

tf.Tensor(b"It may be impossible to get a completely error-free corpus due to the nature of this kind of collaborative effort. However, if we encourage members to contribute sentences in their own languages rather than experiment in languages they are learning, we might be able to minimize errors.\tIl est peut-\xc3\xaatre impossible d'obtenir un Corpus compl\xc3\xa8tement d\xc3\xa9nu\xc3\xa9 de fautes, \xc3\xa9tant donn\xc3\xa9e la nature de ce type d'entreprise collaborative. Cependant, si nous encourageons les membres \xc3\xa0 produire des phrases dans leurs propres langues plut\xc3\xb4t que d'exp\xc3\xa9rimenter dans les langues qu'ils apprennent, nous pourrions \xc3\xaatre en mesure de r\xc3\xa9duire les erreurs.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2024159 (CK) & #2024564 (sacredceltic)", shape=(), dtype=string)
106


## Data Preprocessing

### Converting text to vectors

In [ ]:
# Vectorizing english sequence - token index method
english_Vector=tf.keras.layers.TextVectorization(max_tokens=10000,
                                                 standardize="lower_and_strip_punctuation",
                                                 output_sequence_length=70
                                                 )

In [ ]:
# Vectorizing french sequence - token index method
french_Vector=tf.keras.layers.TextVectorization(max_tokens=10000,
                                                 standardize="lower_and_strip_punctuation",
                                                 output_sequence_length=70
                                                 )

### Getting Vocabulary

In [ ]:
#for getting all vocabulary
def vocab(inp):
  text=tf.strings.split(inp,"\t")
  return text[0],"start "+text[1]+" end"

In [ ]:
vocab_text=dataset.map(vocab)

In [ ]:
english=vocab_text.map(lambda x,y:x)
french=vocab_text.map(lambda x,y:y)

In [ ]:
#getting vocabulary for vector layer
english_Vector.adapt(english)
french_Vector.adapt(french)

### Vectorizing

In [ ]:
# model should have one input for encoder (english), one input for decoder (french shifted right), one output for decoder (french w/ end token)
def in_out(text):
  split=tf.strings.split(text, "\t")
  return split[0],"start "+split[1],split[1]+" end"


In [ ]:
inp_out=dataset.map(in_out)

In [ ]:
def vectorizer(inp_enc,inp_dec,out_dec):
  return english_Vector(inp_enc),french_Vector(inp_dec),french_Vector(out_dec)

In [ ]:
data=inp_out.map(vectorizer)

In [ ]:
for i,j,k in data.take(1):
  print(i,j,k)

tf.Tensor(
[44  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0], shape=(70,), dtype=int64) tf.Tensor(
[  2 104   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0], shape=(70,), dtype=int64) tf.Tensor(
[104   3   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0], shape=(70,), dtype=int64)


In [ ]:
english_Vector.get_vocabulary()[44]

'go'

In [ ]:
french_Vector.get_vocabulary()[104]

'va'

In [ ]:
# Data preparation for model
def vectorizer_model(inp_enc,inp_dec,out_dec):
  return (english_Vector(inp_enc),french_Vector(inp_dec)),french_Vector(out_dec)

In [ ]:
required_data=inp_out.map(vectorizer_model)

In [ ]:
for i,j in required_data.take(1):
  print(i)

(<tf.Tensor: shape=(70,), dtype=int64, numpy=
array([44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0])>, <tf.Tensor: shape=(70,), dtype=int64, numpy=
array([  2, 104,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0])>)


In [ ]:
req_data_1=required_data.shuffle(buffer_size=3000).batch(64).prefetch(buffer_size=tf.data.AUTOTUNE)
number_of_batches=217975/64
training_data_1=req_data_1.take(int(0.8*number_of_batches))
val_data_buf=req_data_1.skip(int(0.8*number_of_batches))
val_data_1=val_data_buf.take(int(0.1*number_of_batches))
test_data_1=val_data_1.skip(int(0.1*number_of_batches))

In [ ]:
for i,j in training_data_1.take(1):
  print(i)

(<tf.Tensor: shape=(64, 70), dtype=int64, numpy=
array([[ 208,   16,    0, ...,    0,    0,    0],
       [  27, 3624,    0, ...,    0,    0,    0],
       [ 124,   65,    0, ...,    0,    0,    0],
       ...,
       [ 524,   13,    0, ...,    0,    0,    0],
       [  21, 3566,    0, ...,    0,    0,    0],
       [ 356,   12,    0, ...,    0,    0,    0]])>, <tf.Tensor: shape=(64, 70), dtype=int64, numpy=
array([[   2, 2250,    0, ...,    0,    0,    0],
       [   2,  262,    1, ...,    0,    0,    0],
       [   2,  165, 6822, ...,    0,    0,    0],
       ...,
       [   2, 4529,   35, ...,    0,    0,    0],
       [   2,    4,   25, ...,    0,    0,    0],
       [   2,  920,  649, ...,    0,    0,    0]])>)


In [ ]:
req_data=required_data.shuffle(buffer_size=3000).unbatch().batch(50000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
number_of_batches=217975/5000

In [ ]:
training_data=req_data.take(int(0.8*number_of_batches))
val_data=req_data.skip(int(0.8*number_of_batches))

In [ ]:
for i,j in training_data.take(1):
  print(print(i[0]))

tf.Tensor([ 17 539   0 ...   0   0   0], shape=(50000,), dtype=int64)
None


## Model Building

In [ ]:
# encoder
input_1=tf.keras.layers.Input(shape=(70))
embedding=tf.keras.layers.Embedding(10000,(50))(input_1)
encoded_output=tf.keras.layers.LSTM(3)(embedding)

In [ ]:
# decoder
input_dec=tf.keras.layers.Input(shape=(70,))
embedding_dec=tf.keras.layers.Embedding(10000,50)(input_dec)
decoded_output=tf.keras.layers.GRU(3,return_sequences=True)(embedding_dec,initial_state=encoded_output)

In [ ]:
# fully connected
fc=tf.keras.layers.Dense(10000,activation="softmax")(decoded_output)

In [ ]:
# Translation
translation=tf.keras.Model([input_1,input_dec],fc)

In [ ]:
translation.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 70)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 70)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 70, 50)       500000      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 70, 50)       500000      ['input_2[0][0]']                
                                                                                              

In [ ]:
translation.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5e-4),)

In [ ]:
translation.fit(training_data_1,validation_data=val_data_1,epochs=5)

Epoch 1/5
    260/Unknown - 261s 976ms/step - loss: 8.6460

In [ ]:
index_to_word={x:y for x, y in zip(range(len(french_Vector.get_vocabulary())),
                                   french_Vector.get_vocabulary())}

In [ ]:
def translator(english_sent):
  eng_vector=english_Vector([english_sent])

  dec_inp="start"
  for i in range(70):
    fre_vector=french_Vector([dec_inp])

    output=translation.predict([eng_vector,fre_vector])

    word=tf.argmax(output,axis=-1)[0][i].numpy()
    french_word=index_to_word[word]
    if french_word=="end":
      break
    dec_inp+=' '+french_word
  return dec_inp

In [ ]:
translator("i read book")

In [ ]:
english_Vector(["i go read"])